In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten

C:\Users\I353565\AppData\Local\Anaconda\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_zf_net():
    inputShape = (225,225,3)  # Note that in paper the input size mentioned is 224x224. But that gives calculation errors
                              # floor((224-7)/2) + 1 = 109
                              # With 225, it gives exact 110. In Pytorch inbuilt implementation, it uses 224 as input size but uses a padding of 2
    zfnet = Sequential()
    # Layer 1.
        # [Input] ==> 225x225x3
        # --> 225x225x3 ==> [Convolution: size=(7x7x3)x96, strides=2, padding=valid] ==> 110x110x96
        # --> 110x110x96 ==> [ReLU] ==> 110x110x96
        # --> 110x110x96 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 55x55x96
        # --> [Output] ==> 55x55x96
    zfnet.add(Conv2D(filters = 96, kernel_size = (7,7), strides = (2,2), input_shape = inputShape, padding = 'valid'))
    zfnet.add(Activation('relu'))
    zfnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    # Layer 2.
        # [Input] ==> 55x55x96
        # --> 55x55x96 ==> [Convolution: size=(5x5x96)x256, strides=2, padding=valid] ==> 26x26x256
        # --> 26x26x256 ==> [ReLU] ==> 26x26x256
        # --> 26x26x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 13x13x256 !!!Calculation error here too. (26-3)/2 doesn't give a whole number
        # --> [Output] ==> 13x13x256
    zfnet.add(Conv2D(filters = 256, kernel_size = (5,5), strides = (2,2), padding = 'valid'))
    zfnet.add(Activation('relu'))
    zfnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    # Layer 3.
        # [Input] ==> 13x13x256
        # --> 13x13x256 ==> [Convolution: size=(3x3x256)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    zfnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))       #Same as AlexNet
    zfnet.add(Activation('relu'))
    
      # Layer 4.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    zfnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))
    zfnet.add(Activation('relu'))
    
    # Layer 5.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x256, strides=1, padding=same] ==> 13x13x256
        # --> 13x13x256 ==> [ReLU] ==> 13x13x256
        # --> 13x13x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 6x6x256
        # --> [Output] ==> 6x6x256
    zfnet.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
    zfnet.add(Activation('relu'))
    zfnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 6.
        # [Input] ==> 6x6x256=9216
        # --> 9216 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> [Output] ==> 4096
    zfnet.add(Flatten())
    zfnet.add(Dense(4096))
    zfnet.add(Activation("relu"))
   
    # Layer 7.
        # [Input] ==> 4096
        # --> 4096 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> [Output] ==> 4096
    
    zfnet.add(Dense(4096))
    zfnet.add(Activation("relu"))
    
    # Layer 8.
        # [Input] ==> 4096
        # --> 4096 ==> [Logits: neurons=1000] ==> 1000
        # --> [Output] ==> 1000
    zfnet.add(Dense(10))  #Assuming number of classes is 10
    zfnet.add(Activation("softmax"))
    
    zfnet.summary()
    return zfnet
    

In [3]:
model = create_zf_net()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 110, 110, 96)      14208     
_________________________________________________________________
activation_1 (Activation)    (None, 110, 110, 96)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 256)       614656    
_________________________________________________________________
activation_2 (Activation)    (None, 25, 25, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 384)       885120    
__________